<a href="https://colab.research.google.com/github/shelly-serafimovich/Data-analysis-and-presentation-laboratory---HW1---00094295--Part-3-Pinecone-VectorDB-and-RAG/blob/main/Lab_HW1_O3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [1]:
!pip install transformers sentence-transformers datasets cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
!pip install pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 3.8 MB/s eta 0:00:00


In [3]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 3.8 MB/s eta 0:00:00


In [4]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from pinecone import Pinecone, ServerlessSpec
import os
from tqdm import tqdm
import cohere
import numpy as np
import warnings
from IPython.display import display
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


#  Q3 - RAG



#### RAG Pipeline:


* An embedding model (For example, a sentence-transformer)

* A vector database, we will use the free-to-use Pinecone API (limited to 100k vectors)

* An LLM to chat with, we will use the CohereChat API (Similar to OpenAI's chatGPT, but free)

#####  APIs

In [5]:
COHERE_API_KEY = 'pvGT5Qid1LtwyhTLw9woGMUxn3vDrUhUg4EOZriA'
PINECONE_API_KEY = '218218aa-bfa0-4c49-9bc9-651ed4b7402f'

## First Element - Embedding Model

In [6]:
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL = 'all-MiniLM-L6-v2'
model = SentenceTransformer(EMBEDDING_MODEL)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
def load_and_embedd_dataset(
        dataset_name: str = 'fancyzhx/ag_news',
        split: str = 'train',
        model: SentenceTransformer = SentenceTransformer('all-MiniLM-L6-v2'),
        text_field: str = 'text',
        rec_num: int = 400
) -> tuple:
    """
    Load a dataset and embedd the text field using a sentence-transformer model
    Args:
        dataset_name: The name of the dataset to load
        split: The split of the dataset to load
        model: The model to use for embedding
        text_field: The field in the dataset that contains the text
        rec_num: The number of records to load and embedd
    Returns:
        tuple: A tuple containing the dataset and the embeddings
    """
    from datasets import load_dataset
    print("Loading and embedding the dataset")

    # Load the dataset
    dataset = load_dataset("fancyzhx/ag_news", split=split)
    dataset = dataset.filter(lambda x: x['label'] == 1)
    # Embed the first `rec_num` rows of the dataset
    embeddings = model.encode(dataset[text_field][:rec_num])

    print("Done!")
    return dataset, embeddings

In [8]:
DATASET_NAME = 'fancyzhx/ag_news'

dataset, embeddings = load_and_embedd_dataset(
    dataset_name=DATASET_NAME,
    rec_num=30000,
    model=model,
)
shape = embeddings.shape

Loading and embedding the dataset


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

Done!


Lets us look at the dataset and the embeddings

In [9]:
pd_dataset = dataset.to_pandas()
pd_dataset

,text,label
0,"Phelps, Thorpe Advance in 200 Freestyle (AP) A...",1
1,Reds Knock Padres Out of Wild-Card Lead (AP) A...,1
2,"Dreaming done, NBA stars awaken to harsh Olymp...",1
3,"Indians Beat Twins 7-1, Nearing AL Lead (AP) A...",1
4,"Galaxy, Crew Play to 0-0 Tie (AP) AP - Kevin H...",1
...,...,...
29995,Rauffer Beats Favorites to Win Downhill VAL G...,1
29996,Renteria signing a top-shelf deal Red Sox gene...,1
29997,Saban not going to Dolphins yet The Miami Dolp...,1
29998,Today's NFL games PITTSBURGH at NY GIANTS Time...,1


In [10]:
print(f"The embeddings shape: {embeddings.shape}")

The embeddings shape: (30000, 384)


## Second Element - Vector Database
We will use Pinecone's free-to-use vectorDB

In [11]:
def create_pinecone_index(
        index_name: str,
        dimension: int,
        metric: str = 'cosine'
):
    """
    Create a pinecone index if it does not exist
    Args:
        index_name: The name of the index
        dimension: The dimension of the index
        metric: The metric to use for the index
    Returns:
        Pinecone: A pinecone object which can later be used for upserting vectors and connecting to VectorDBs
    """
    from pinecone import Pinecone, ServerlessSpec
    print("Creating a Pinecone index...")
    pc = Pinecone(api_key=PINECONE_API_KEY)
    existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
    if index_name not in existing_indexes:
        print(f"Index {index_name} does not exist. Creating...")
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric=metric,
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )
    print("Done!")
    return pc

In [18]:
INDEX_NAME = 'hnsw-index'

# Create the vector database
# We are passing the index_name and the size of our embeddings
pc = create_pinecone_index(INDEX_NAME, shape[1])

Creating a Pinecone index...
Done!


Now that we have created the vector database, let's add some data to it!

In [12]:
def upsert_vectors(
        index: Pinecone,
        embeddings: np.ndarray,
        dataset: dict,
        text_field: str = 'text',
        batch_size: int = 128
):
    """
    Upsert vectors to a pinecone index
    Args:
        index: The pinecone index object
        embeddings: The embeddings to upsert
        dataset: The dataset containing the metadata
        batch_size: The batch size to use for upserting
    Returns:
        An updated pinecone index
    """
    print("Upserting the embeddings to the Pinecone index...")
    shape = embeddings.shape

    ids = [str(i) for i in range(shape[0])]
    meta = [{text_field: text} for text in dataset[text_field]]

    # create list of (id, vector, metadata) tuples to be upserted
    to_upsert = list(zip(ids, embeddings, meta))

    for i in tqdm(range(0, shape[0], batch_size)):
        i_end = min(i + batch_size, shape[0])
        index.upsert(vectors=to_upsert[i:i_end])
    return index



In [16]:
import pinecone as pc

In [19]:
# Upsert the embeddings to the Pinecone index
index = pc.Index(INDEX_NAME)
index_upserted = upsert_vectors(index, embeddings, dataset)

Upserting the embeddings to the Pinecone index...


100%|██████████| 235/235 [01:59<00:00,  1.96it/s]


Let's view the index statistics!

In [20]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 30000}},
 'total_vector_count': 30000}

## Third Element - LLM
We will use [Cohere's chat API](https://cohere.com/chat)

1. query = "Who are the Greek 200 meter sprinters who retired from the Olympic Games in Athens?"

2. query = "In a friendly soccer match between Germany and Cameroon, what was the result of the match?"

3. query = "In the 2005-2006 season, the Cleveland Cavaliers rejected the contract of the player Dajuan Wagner, who became a free agent, what was the amount of the contract?"


In [26]:
import cohere

# Define the list of queries
queries = [
    "Who are the Greek 200 meter sprinters who retired from the Olympic Games in Athens?",
    "In a friendly soccer match between Germany and Cameroon, what was the result of the match?",
    "In the 2005-2006 season, the Cleveland Cavaliers rejected the contract of the player Dajuan Wagner, who became a free agent, what was the amount of the contract?"
]

co = cohere.Client(api_key=COHERE_API_KEY)
# Loop through each query and get the response
for query in queries:
    response = co.chat(
        model='command-r-plus',
        message=query,
    )
    print(f"Query: {query}")
    print(f"Response: {response.text}\n")
    print("-------------------------------------------------------")
    print("\n")

Query: Who are the Greek 200 meter sprinters who retired from the Olympic Games in Athens?
Response: I cannot find any specific information on Greek 200-meter sprinters who retired from the Olympic Games in Athens. However, I can provide some general information on Greek athletes in that event at the Athens Olympics.

At the 2004 Athens Olympic Games, the Greek athletes who competed in the men's 200-meter sprint event were Konstantinos Kenteris and Anastasios Gousis. Kenteris was the reigning Olympic champion from the 2000 Sydney Games but failed to defend his title in Athens. Both sprinters progressed to the semifinals, but neither advanced to the final round.

It is worth noting that Konstantinos Kenteris and his training partner Ekaterini Thanou withdrew from the Athens Olympics amid a doping scandal, but they did not officially retire. They were involved in a controversial incident where they missed multiple drug tests and eventually withdrew from the Games.

If you have additional

 queries vectors from the database

In [23]:
def augment_prompt(
        query: str,
        model: SentenceTransformer = SentenceTransformer('all-MiniLM-L6-v2'),
        index=None,
) -> str:
    """
    Augment the prompt with the top 3 results from the knowledge base
    Args:
        query: The query to augment
        index: The vectorstore object
    Returns:
        str: The augmented prompt
    """
    results = [float(val) for val in list(model.encode(query))]

    # get top k results from knowledge base
    query_results = index.query(
        vector=results,
        top_k=5,
        include_values=True,
        include_metadata=True
    )['matches']
    text_matches = [match['metadata']['text'] for match in query_results]

    # get the text from the results
    source_knowledge = "\n\n".join(text_matches)

    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.
    Contexts:
    {source_knowledge}
    If the answer is not included in the source knowledge - say that you don't know.
    Query: {query}"""
    return augmented_prompt, source_knowledge

In [25]:
# Let us remember our query
for query in queries:
  augmented_prompt, source_knowledge = augment_prompt(query, model=model, index=index)
  response = co.chat(
          model='command-r-plus',
          message=augmented_prompt,
      )

  print(f"Query: {query}")
  print(f"Response: {response.text}\n")
  print(source_knowledge)
  print("-------------------------------------------------------")
  print("\n")

Query: Who are the Greek 200 meter sprinters who retired from the Olympic Games in Athens?
Response: The Greek sprinters who withdrew from the 2004 Olympic Games in Athens were Kostas Kenteris and Katerina Thanou.

Withdrawal for Greek sprinters ATHENS -- Two of Greece #39;s most famous athletes and their coach made their first public appearance here yesterday only to withdraw from the Olympic Games rather than face a panel charged with looking into a ...

Greek sprinters withdraw from Olympics ATHENS -- The final curtain fell Wednesday on a week-long drama that has riveted this nation when Greece #39;s two top sprinters, swept up in a suspected doping scandal, withdrew from the Summer Olympics and competitions that might have ...

Greek Sprinters Given Two-Day Reprieve  ATHENS (Reuters) - Greek sprinters Costas Kenteris and  Katerina Thanou won a two-day reprieve at the Athens Olympics  Monday so they could appear in person at a disciplinary hearing  into their missed drugs tests and 